In [1]:
import os
from classesCHANGE import PointCloudFiltering, UnsupervisedSegmentationAlgorithm, PointCloudGenerator, CropAnalyzer
import time
import plyfile 
import open3d as o3d
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Instantiate classes
pcd_filtering = PointCloudFiltering()
unsup_segmentation = UnsupervisedSegmentationAlgorithm()
pcd_functions = PointCloudGenerator(r"C:/Users/Jay")


In [3]:
pcd = o3d.io.read_point_cloud(r'C:\Users\Jay\Documents\Github\HarvestEye\main\avoFixed.ply')
o3d.visualization.draw_geometries([pcd])

In [4]:
# Downsample the point cloud
voxel_size = 0.2
downsampled_pcd = pcd_functions.downsamples_pointcloud(pcd, voxel_size)

# Normalize the point cloud
xyzrgb_normalized = pcd_functions.pcd_array_normalized(downsampled_pcd)

# Define weights and number of clusters for KMeans
weights = [5, 5, 500, 1, 550, 1]
num_clusters = 2
kmeans_labels, kmeans = unsup_segmentation.normalized_and_weighted_kmeans(weights, xyzrgb_normalized, num_clusters)

# Prepare cluster colors array
cluster_colors = np.zeros_like(xyzrgb_normalized[:, :3])  # Use only the XYZ part for colors

# Define colors for clusters
colors = {
    "green": [0, 150/255, 0],  # Green color
    "brown": [0.647, 0.165, 0.165]  # Brown color
}

# Identify the larger cluster
larger_cluster_label = np.argmax(np.bincount(kmeans_labels))

# Assign colors based on cluster sizes
for i in range(num_clusters):
    color = colors["brown"] if i == larger_cluster_label else colors["green"]
    cluster_colors[kmeans_labels == i] = color

seg_pcd = downsampled_pcd
seg_pcd.colors = o3d.utility.Vector3dVector(cluster_colors)
o3d.visualization.draw_geometries([seg_pcd])

In [5]:
ground_points = np.asarray(seg_pcd.points)[kmeans_labels == larger_cluster_label]
crop_points = np.asarray(seg_pcd.points)[kmeans_labels != larger_cluster_label]

# Filter the Ground Points list
pcd_filtering.filter_ground_points()

# Filter the Crop Points via a grid
pcd_filtering.filter_points_by_grid()

# Filter the resulting DBSCAN by min and max cluster sizes
pcd_filtering.dbscan_cluster_filtering()

crop_points_xy = crop_points[:, :2]
# Filter points based on DBSCAN clustering
eps = 0.2  # epsilon value for DBSCAN
dbscan_labels, clustered_points_xyz = unsup_segmentation.crop_clustering_dbscan(eps, crop_points_xy)

#Now, retrieve the Z coordinates for clustered tree points
clustered_crop_indices = np.where(dbscan_labels != -1)[0]  # Indices of points that are not noise
#Now, retrieve the Z coordinates for clustered tree points
clustered_tree_points = crop_points[clustered_crop_indices]

import random
#Convert clustered_tree_points to an Open3D point cloud
dbscan_clusters_pcd = o3d.geometry.PointCloud()
dbscan_clusters_pcd.points = o3d.utility.Vector3dVector(clustered_tree_points)

# Assign random colors to each cluster
cluster_colors = {}
for label in np.unique(dbscan_labels):
    if label == -1:  # Skip noise points
        continue
    color = [random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1)]  # Generate a random RGB color
    cluster_colors[label] = color

# Assign colors to each point based on its cluster
point_colors = []
for label in dbscan_labels:
    if label == -1:  # Noise points
        point_colors.append([0.5, 0.5, 0.5])  # Gray color
    else:
        point_colors.append(cluster_colors[label])

# Set colors for the clustered tree points
dbscan_clusters_pcd.colors = o3d.utility.Vector3dVector(point_colors)

# Set the visualization mode to "unlit" for precise color rendering
o3d.visualization.draw_geometries([dbscan_clusters_pcd])# + [pcd])



In [ ]:
crop_analyzer = CropAnalyzer()


